In [2]:
import openai

import os
from pathlib import Path
import random
import time
import json

In [3]:
openai.api_key = 'Enter your OpenAI API key'

In [4]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(path, obj):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(obj, f, indent=4, ensure_ascii=False)

In [4]:
def generate(messages, **kwargs):
    openai.api_key = openai.api_key or os.getenv("OPENAI_API_KEY")

    while True:
        try:
            res = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=messages,
                **kwargs
            )
            return res.choices
        except openai.error.RateLimitError:
            time.sleep(1)
        except openai.error.InvalidRequestError as e:
            return e

In [5]:
class GPTMessage:
    def __init__(self):
        self.messages = []

    def add_system(self, msg):
        self.messages.append({'role': 'system', 'content': msg})

    def add_user(self, msg):
        self.messages.append({'role': 'user', 'content': msg})

    def add_assistant(self, msg):
        self.messages.append({'role': 'assistant', 'content': msg})

    def to_json(self):
        return json.dumps(self.messages, ensure_ascii=False)

    def get_messages(self):
        return self.messages

    def get_text(self):
        return '\n\n'.join([x['content'] for x in self.messages])

    def get_answer(self):
        res = generate(self.messages)
        return res[0]['message']['content']

In [5]:
msg = GPTMessage()
msg.add_system('You are the algorithm comparator who figures out the main differences between two given python code, and determine the main algorithms of two codes is same enough or not. If the time complexity is same, you can judge them as similar algorithm but different approaching. Else if their time complexities is different, you can judge them as different algorithm. You can compare them line by line, but you must compare abstractedly, not much in detail, so that I can compare them in terms of their main algorithm which the codes want to implement. ')
msg.add_user('''Here is the first code:

def find(visited, graph, node):
    if node not in visited:
        print (node)
        visited.add(node)
        for neighbour in graph[node]:
            find(visited, graph, neighbour)''')
msg.add_user('''Here is the second code:

def find(visited, graph, node):
  queue = []

  visited.append(node)
  queue.append(node)

  while queue:
    m = queue.pop(0)
    print (m, end = " ")

    for neighbour in graph[m]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)''')

In [67]:
res = msg.get_answer()

In [68]:
print(res)

Both of the codes implement the same algorithm of finding and printing all the nodes of a graph. However, the first code uses a recursive approach while the second code uses an iterative approach. The time complexity is also the same, i.e., O(V+E) where V is the number of vertices and E is the number of edges in the graph. Therefore, both codes implement similar algorithms using different approaches.


In [6]:
class CodeComparator(GPTMessage):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

        self._build()

    def _build(self):
        self.add_user('You are the algorithm comparator who figures out the main differences between two given python code, and determine the main algorithms of two codes is same enough or not. You need to tell me the name of the algorithm used, if exists. If the time complexity is same, you can judge them as similar algorithm but different approaching. Else if their time complexities is different, you can judge them as different algorithm. You can compare them line by line, but you must compare in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.')
        self.add_user(f'\nHere is the first code:\n\n{self.first}')
        self.add_user(f'\nHere is the second code:\n\n{self.second}')

In [10]:
dfs_code = '''def find(visited, graph, node):
    if node not in visited:
        print (node)
        visited.add(node)
        for neighbour in graph[node]:
            find(visited, graph, neighbour)'''
bfs_code = '''def find(visited, graph, node):
  queue = []

  visited.append(node)
  queue.append(node)

  while queue:
    m = queue.pop(0)
    print (m, end = " ")

    for neighbour in graph[m]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)'''

dijkstra_code = '''import heapq

def find(graph, start):
  distances = {node: float('inf') for node in graph}
  distances[start] = 0
  queue = []
  heapq.heappush(queue, [distances[start], start])

  while queue:
    current_distance, current_destination = heapq.heappop(queue)
    if distances[current_destination] < current_distance:
      continue

    for new_destination, new_distance in graph[current_destination].items():
      distance = current_distance + new_distance
      if distance < distances[new_destination]:
        distances[new_destination] = distance
        heapq.heappush(queue, [distance, new_destination])

  return distances'''

hfseq_wrong = '''t=int(input())
for i in range(t):
    n=int(input())
    list1=list(map(int,input().split()))
    if(n%2==0):
        print("yes")
    else:
        print("no")'''

hfseq_correct = '''from math import ceil, gcd
from functools import reduce
for _ in range(int(input())):
    n = int(input());a = list(map(int, input().split()));size = ceil(n / 2);multiples_2 = [v // 2 for v in a if v % 2 == 0]
    if len(multiples_2) < size or reduce(gcd, multiples_2) > 1:print('NO')
    else:
        triats, g = [multiples_2[0]], multiples_2[0]
        for v in multiples_2:
            nou_g = gcd(g, v)
            if nou_g < g:
                triats.append(v);g = nou_g
                if g == 1:break
        print('YES') if len(triats) <= size else print('NO')'''

In [80]:
comparator = CodeComparator(hfseq_correct, hfseq_wrong)
comparator.get_messages()

[{'role': 'user',
  'content': 'You are the algorithm comparator who figures out the main differences between two given python code, and determine the main algorithms of two codes is same enough or not. You need to tell me the name of the algorithm used, if exists. If the time complexity is same, you can judge them as similar algorithm but different approaching. Else if their time complexities is different, you can judge them as different algorithm. You can compare them line by line, but you must compare in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.'},
 {'role': 'user',
  'content': "\nHere is the first code:\n\nfrom math import ceil, gcd\nfrom functools import reduce\nfor _ in range(int(input())):\n    n = int(input());a = list(map(int, input().split()));size = ceil(n / 2);multiples_2 = [v // 2 for v in a if v % 2 == 0]\n    if len(multiples_2) < size or reduce(gcd, multiples_2) > 1:print('NO')\n    else:\n        t

In [81]:
res = comparator.get_answer()
print(res)

The first code uses the algorithm called "Extended Euclidean Algorithm" to find the greatest common divisor of given numbers. It also uses some logic to perform operations in a list.

The second code checks whether the input array has even number of elements or not. Hence, it doesn't have any specific algorithm.

Therefore, the time complexities of these two codes are not comparable as they solve different problems.


In [7]:
class PseudoCodeGenerator(GPTMessage):
    def __init__(self, code):
        super().__init__()
        self.code = code

        self._build()

    def _build(self):
        self.add_user('You are the pseudo-code generator who generate pseudo code in mathematical notation, ususally used in the academic paper, in ascii format, not latex. I will give you the function of the python code, which has input and output, thus you can translate the main algorithm of the code as pseudo-code. You can use english word on the abstracted features in the pseudo code, but you should not use a python code or other language directly, and should not generate any comments. Also, you need to generate pseudo code abstractive enough, so that I can understand the whole algorithm without looking inside the details of the implementation. You can replace the function name as an english words if the function name is too specific to some libraries, but the replaced version also needs to be abstractive enough. You can skip the original function name. Also, You can change variable names shorter, such as an capital letter.')
        self.add_user(f'\nHere is the python code:\n\n{self.code}')

In [12]:
pseudo_gen = PseudoCodeGenerator(hfseq_correct)
pseudo_gen.get_messages()

[{'role': 'user',
  'content': 'You are the pseudo-code generator who generate pseudo code in mathematical notation, ususally used in the academic paper, in ascii format, not latex. I will give you the function of the python code, which has input and output, thus you can translate the main algorithm of the code as pseudo-code. You can use english word on the abstracted features in the pseudo code, but you should not use a python code or other language directly, and should not generate any comments. Also, you need to generate pseudo code abstractive enough, so that I can understand the whole algorithm without looking inside the details of the implementation. You can replace the function name as an english words if the function name is too specific to some libraries, but the replaced version also needs to be abstractive enough. You can skip the original function name. Also, You can change variable names shorter, such as an capital letter.'},
 {'role': 'user',
  'content': "\nHere is the 

In [94]:
res = pseudo_gen.get_answer()
print(res)



1. Initialize ceil and gcd functions from math library and reduce function from functools library  # The implementation imports some libraries

2. Take input for the number of test cases (t) # The implementation asks for the number of cases.

3. For each test case perform the following steps:
    
    a. Take input for the size of the array (n), list of integers(a)    #The implementation takes input for the size of array and a list of integers
    
    b. Calculate the value of size as ceil(n/2) i.e. size = ceil(n / 2)    #Implementation calculates size as ceil(n/2) 
    
    c. Create a list multiples_2, which will have elements of a if they are divisible by 2, divided by 2, else ignore.        #The implementation creates a list multiples_2, containing elements of a that are divisible by 2, divided by 2, else ignore.
    
    d. Check if length of multiples_2 is less than size or if g.c.d. of elements in multiples_2 is greater than 1. If any of this condition is True, print 'NO'. El

In [97]:
'''You are the pseudo-code generator who generate pseudo code in mathematical notation, usually used in the academic paper.

I will give you the function of the python code, which has input and output, thus you can translate the main algorithm of the code as a pseudo-code.

- You must generate pseudo-code only, not generate additional note or explanation. But your pseudo-code must be explainable.
- You can use english words on the abstracted features in the pseudo code, and you can replace some function names to english words if the function name is too specific on the libraries. But these replaced names must be abstractive enough.
- You cannot generate code as python or other languages directly, and use mathematical notation instead.
- Your result need to be short as much as possible, so you can replace the variable name as an capital letter, with explanation about substitution.
- The result must be abstractive enough, so that I can understand the whole algorithm only with the generated pseudo-code.
- You can keep the operator symbols as-is, such as '<', '>=', '+', '//'.'''

'You are the pseudo-code generator who generate pseudo code in mathematical notation, ususally used in the academic paper, in ascii format, not latex.\n\nI will give you the function of the python code, which has input and output, thus you can translate the main algorithm of the code as pseudo-code.\n\nYou can use english word on the abstracted features in the pseudo code, but you should not use a python code or other language directly, and should not generate any comments. Also, you need to generate pseudo code abstractive enough, so that I can understand the whole algorithm without looking inside the details of the implementation. You can replace the function name as an english words if the function name is too specific to some libraries, but the replaced version also needs to be abstractive enough. You can skip the original function name. Also, You can change variable names shorter, such as an capital letter.'

In [ ]:
'''You are the algorithm comparator who figures out the main differences between two given python code, and determine the main algorithms of two codes is same enough or not.

You need to tell me the name of the algorithm used, if exists. If the time complexity is same, you can judge them as similar algorithm but different approaching. Else if their time complexities is different, you can judge them as different algorithm. You can compare them line by line, but you must compare in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.'''

In [ ]:
'''You are the algorithm comparator who figures out the main differences of given solution code from the answer code, and determine the main algorithms of two codes is same enough or not.
You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- If the time complexity of the solution code is same, you can judge them as similar algorithm. Although their time complexities are same, they have different approaching if their algorithms are different.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you must compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.'''

In [ ]:
'''You are the algorithm comparator who figures out the main differences of given solution code from the answer code, who are correct to solve given problem description. You need to determine whether the two algorithms are same so that the solution code is success to solve the problem, or not. If the solution code fails, you must specify what differences makes the solution code failed to solve the problem, comparing the answer code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- If the time complexity of the solution code is same, you can judge them as similar algorithm. Although their time complexities are same, they have different approaching if their algorithms are different.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you must compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.
- You cannot describe the algorithm of the answer code directly.'''

In [ ]:
'''You are the algorithm analyzer who figures out the main differences of given wrong solution code from the answer code, who are correct to solve given problem description. You need to figure out the main differences of the algorithm which makes the solution code failed to solve the problem, comparing the answer code. There might be only simple typo error, or simple logical error. Regardless of the size of the differences, you must specify the differences of the wrong solution code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- You must specify the main differences is simple, like a typo error or simple logical error, or complex enough, such as inefficient algorithm or wrong approach, or existence of missing logic.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you need to compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.
- You cannot describe the algorithm of the answer code directly. It must be hidden to viewer.'''

In [ ]:
'''You are the algorithm analyzer who figures out the main differences of given wrong solution code from the answer code, who are correct to solve given problem description. You need to figure out the main differences of the algorithm which makes the solution code failed to solve the problem, comparing the answer code. There might be only simple typo error, or simple logical error. Regardless of the size of the differences, you must specify the differences of the wrong solution code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- You must specify the main differences is simple, like a typo error or simple logical error, or complex enough, such as inefficient algorithm or wrong approach, or existence of missing logic.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you need to compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- When the differences of code lines are given, you must explain the differences of the algorithm considering the differences of code lines, and why that different lines make the different results.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.
- You cannot describe the algorithm of the answer code directly. It must be hidden to viewer.'''

In [8]:
'''You are the test case generator who generates proper test case of given problem description. The problem description contains input format and output format, and the constraints about the values of input and output, and some examples of the test case for the problem. Sometimes the output can vary on the same input, so in this case the output value can be one of the possibilities. I will give you the problem description with the explanation about input & output format, constraints, examples, and other necessary information.

You must follow these instructions when you generate test cases:

- You need to generate at least 3 test cases which has its own input & output.
- Every test case you made must be independent from other test cases.
- The generated test cases should cover the test coverage enough, which means test cases must consider the exceptional cases or special cases (such as values at limit).
- The generated test case must be different from the given test case, and must test other cases such as larger number, exceptional cases, edge cases.
- Every test case must contains valid input and output, which has right format of them same as examples.
- You must not generate invalid test case, for example, which contains an invalid value (which exceeds limitation), or which has not a format the problem describes.
- Output must be exact along with the input. If the output is wrong with given input, the test case is invalid.
- You don't need to generate any explanation about the test case. Just pairs of input & output is good.
'''

"You are the test case generator who generates proper test case of given problem description. The problem description contains input format and output format, and the constraints about the values of input and output, and some examples of the test case for the problem. Sometimes the output can vary on the same input, so in this case the output value can be one of the possibilities. I will give you the problem description with the explanation about input & output format, constraints, examples, and other necessary information.\n\nYou must follow these instructions when you generate test cases:\n\n- You need to generate at least 3 test cases which has its own input & output.\n- Every test case you made must be independent from other test cases.\n- The generated test cases should cover the test coverage enough, which means test cases must consider the exceptional cases or special cases (such as values at limit).\n- The generated test case must be different from the given test case, and must

In [9]:
from chatgpt_wrapper import ChatGPT

In [10]:
class SolutionComparator:
    def __init__(self, bot, desc, solution, answer):
        self.bot = bot
        self.desc = desc
        self.solution = solution
        self.answer = answer

    def _make_prompt(self):
        prompt = f'''You are the algorithm comparator who figures out the main differences of given solution code from the answer code, who are correct to solve given problem description. You need to determine whether the two algorithms are same so that the solution code is success to solve the problem, or not. If the solution code fails, you must specify what differences makes the solution code failed to solve the problem, comparing the answer code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- If the time complexity of the solution code is same, you can judge them as similar algorithm. Although their time complexities are same, they have different approaching if their algorithms are different.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you must compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.
- You cannot describe the algorithm of the answer code directly.

----
Here is the problem description:

{self.desc}

----
Here is submitted solution code:

{self.solution}

----
Here is the answer code:

{self.answer}'''
        return prompt

    def get_result(self):
        response = self.bot.ask(self._make_prompt())
        return response


In [15]:
desc = '''Chef has an array A of size N. Chef wants to choose any subsequence of size exactly ⌈n/2⌉ from the array such that GCD of all the elements in that sequence must be 2. Chef names such a kind of sequence as a half-sequence.

Help Chef to find whether he would be able to select any half-sequence in the given array.

As a reminder,

A subsequence of an array is a sequence that can be derived from the given array by deleting zero or more elements without changing the order of the remaining elements.
GCD stands for Greatest Common Divisor. The greatest common divisor of a subsequence is the largest integer d such that all the numbers in sequence are divisible by d.
⌈x⌉ is the ceiling (round up) operation:
⌈3.5⌉ = 4, ⌈2⌉ = 2.

Input Format
- The first line contains an integer T denoting the number of test cases. The T test cases then follow.
- The first line of each test case contains a single integer N denoting the size of the array.
- The second line of each test case contains N space-separated integers A_1, A_2, ..., A_N denoting the given array.

Output Format
- For each test case, output on one line YES if Chef can find a half-sequence, else print NO. Output is case insensitive.

Constraints
1 <= T <= 20
2 <= N <= 10^5
1 <= A_i <= 10^9
Sum of N over all test cases does not exceed 10^5'''

solution = '''from itertools import combinations
from math import *
for _ in range(int(input())):
    n=int(input())
    tmp=list(map(int,input().split()))
    a=[]
    for i in tmp:
        if i%2==0:
            a.append(i)
    n=(n+1)//2
    if len(a)<n:
        print("NO")
        continue
    comb=combinations(a,n)
    if n<=18:
        ver=False
        for i in comb:
            i=list(i)
            g=i[0]
            for j in range(1,len(i)):
                g=gcd(g,i[j])
            if g==2:
                ver=True
                break
        if ver:
            print("YES")
        else:
            print("NO")
    else:
        g=a[0]
        for i in range(1,len(a)):
            g=gcd(g,a[i])
        if g==2:
            print("YES")
        else:
            print("NO")'''

answer = '''from math import ceil, gcd
from functools import reduce
for _ in range(int(input())):
    n = int(input());a = list(map(int, input().split()));size = ceil(n / 2);multiples_2 = [v // 2 for v in a if v % 2 == 0]
    if len(multiples_2) < size or reduce(gcd, multiples_2) > 1:print('NO')
    else:
        triats, g = [multiples_2[0]], multiples_2[0]
        for v in multiples_2:
            nou_g = gcd(g, v)
            if nou_g < g:
                triats.append(v);g = nou_g
                if g == 1:break
        print('YES') if len(triats) <= size else print('NO')'''

In [108]:
solution_comparator = SolutionComparator(ChatGPT(), desc, solution, answer)
res = solution_comparator.get_result()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [ ]:
res

In [11]:
import json

In [12]:
with open('problem_samples.json', 'r', encoding='utf-8') as f:
    problems = json.load(f)

In [21]:
keys = list(problems.keys())
desc = problems[keys[3]]
print(desc)

Problem
Read problem statements in Hindi, Bengali, Mandarin Chinese, Russian, and Vietnamese as well.
This is not intended to be the tie-break problem.
In the Chefland army, there are M generals (numbered 1 through M) and N soldiers (numbered 1 through N).
The Army has a vertical hierarchy ― a soldier may command many other soldiers (including zero), but each soldier has at most one commander. A soldier y is a subordinate of a soldier x if soldier x is the commander of soldier y or there is a soldier z commanded by soldier x such that soldier y is a subordinate of soldier z. No soldier is commanded by one of their subordinates because every soldier is feared by all of his subordinates. Moreover, all soldiers are subordinates of soldier 1. Let's denote the commander of soldier i by p_i (p_1 = 0 since soldier 1 cannot have a commander).
As in any organisation, the generals have some soldiers they hate. If a general meets a soldier whom he hates, this general becomes angry for the rest of

In [13]:
def post_process_problem(desc):
    headings = {'input', 'output', 'constraints', 'test', 'scoring'}
    lines = desc.split('\n')

    text = []
    for line in lines:
        line_low = line.lower().strip()
        if line_low.startswith('read problem') or line_low.startswith('##'):
            continue
        if line_low.startswith('example') or line_low.startswith('sample') or line_low.startswith('explanation'):
            break
        words = line_low.split()
        if len(words) > 0 and words[0] in headings:
            text.append('\n' + line)
            continue
        text.append(line)

    res = '\n'.join(text)
    return res

In [23]:
print(post_process_problem(desc))

Problem
This is not intended to be the tie-break problem.
In the Chefland army, there are M generals (numbered 1 through M) and N soldiers (numbered 1 through N).
The Army has a vertical hierarchy ― a soldier may command many other soldiers (including zero), but each soldier has at most one commander. A soldier y is a subordinate of a soldier x if soldier x is the commander of soldier y or there is a soldier z commanded by soldier x such that soldier y is a subordinate of soldier z. No soldier is commanded by one of their subordinates because every soldier is feared by all of his subordinates. Moreover, all soldiers are subordinates of soldier 1. Let's denote the commander of soldier i by p_i (p_1 = 0 since soldier 1 cannot have a commander).
As in any organisation, the generals have some soldiers they hate. If a general meets a soldier whom he hates, this general becomes angry for the rest of the day.
The soldiers should be paid some wages over one or more days. For each valid i, sold

In [14]:
import re
latex_pattern = r'\\[^\s]+'

In [15]:
latexs = {}
for _, prob in problems.items():
    lines = prob.split('\n')
    for line in lines:
        patterns = re.findall(latex_pattern, line)
        for ptn in patterns:
            if ptn not in latexs:
                latexs[ptn] = []
            latexs[ptn].append(line)

In [8]:
latexs.keys()

dict_keys(['\\le', '\\leq', '\\cdot', '\\dots,', '\\ldots,', '\\in', '\\{0.1,', '\\}.', '\\ne', '\\lt', '\\sum_{i=1}^M', '\\mathrm{max}(0,', '\\gt', '\\%', '\\neq', '\\lceil', '\\frac{N}{2}', '\\rceil', '\\frac{5}{2}', '\\frac{4}{2}', '\\frac{3}{2}', '\\{(x_1,y_1),', '\\}:', '\\mathrm{min}\\,(X+d,', '\\mathrm{max}(X-d,', '\\mathrm{max}\\,(X-d,', '\\oplus', '\\lfloor', '\\rfloor', '\\rfloor).', '\\geq', '\\mod{N}.', '\\mod', '\\bmod', '\\mathrm{min}(l_p,', '\\mathrm{min}(3,5)^2', '\\ldots+|W_N|', '\\xrightarrow{+1}', '\\xrightarrow{+2}'])

In [9]:
latexs['\\sum_{i=1}^M']

["Let's define the pretty value of Chef's music score V = \\sum_{i=1}^M K_i \\cdot C_i, where K_i denotes the number of times the i-th pretty melody occurs in Chef's music score as a substring (these occurrences may overlap). Your goal is to maximise the pretty value of Chef's music score."]

In [16]:
def replace_latex(desc):
    replace_table = {
        r'\\bmod': 'mod',
        r'\\cdot': '·',
        r'\\dots': '...',
        r'\\geq': '>=',
        r'\\gt': '>',
        r'\\in': 'in',
        r'\\lceil': '⌈',
        r'\\ldots': '...',
        r'\\leq': '<=',
        r'\\le': '<=',
        r'\\lfloor': '⌊',
        r'\\lt': '<',
        r'\\mathrm{max}': 'Max',
        r'\\mathrm{min}': 'Min',
        r'\\mod{N}': '(mod N)',
        r'\\mod': 'mod',
        r'\\neq': '!=',
        r'\\ne': '!=',
        r'\\oplus': '⊕',
        r'\\rceil': '⌉',
        r'\\rfloor': '⌋',
        r'\\sum': 'Sum',
        r'\\{': '{',
        r'\\}': '}',
        r'\\%': '%',
        r'\\,': ' ',
    }
    frac_pattern = r'\\frac{(\S+)}{(\S+)}'
    arrow_pattern = r'\\xrightarrow{(\S+)}'
    latex_pattern = r'\$([^$]+)\$'
    desc = re.sub(frac_pattern, r'(\g<1> / \g<2>)', desc)
    desc = re.sub(arrow_pattern, r'(-> \g<1>)', desc)
    desc = re.sub(latex_pattern, r'\g<1>', desc)
    for ptn, rpl in replace_table.items():
        desc = re.sub(ptn, rpl, desc)
    return desc

In [11]:
print(replace_latex(post_process_problem(desc)))

Problem
This is not intended to be the tie-break problem.
In the Chefland army, there are M generals (numbered 1 through M) and N soldiers (numbered 1 through N).
The Army has a vertical hierarchy ― a soldier may command many other soldiers (including zero), but each soldier has at most one commander. A soldier y is a subordinate of a soldier x if soldier x is the commander of soldier y or there is a soldier z commanded by soldier x such that soldier y is a subordinate of soldier z. No soldier is commanded by one of their subordinates because every soldier is feared by all of his subordinates. Moreover, all soldiers are subordinates of soldier 1. Let's denote the commander of soldier i by p_i (p_1 = 0 since soldier 1 cannot have a commander).
As in any organisation, the generals have some soldiers they hate. If a general meets a soldier whom he hates, this general becomes angry for the rest of the day.
The soldiers should be paid some wages over one or more days. For each valid i, sold

In [12]:
problems_after = {k: replace_latex(post_process_problem(v)) for k, v in problems.items()}

In [13]:
for pid, desc in problems_after.items():
    print(f'[{pid}]\n')
    print(desc)
    print('\n----\n')

[CATSRATS]

Problem
There are N cats (numbered 1 through N) and M rats (numbered 1 through M) on a line. Each cat and each rat wants to move from some point to some (possibly the same) point on this line. Naturally, the cats also want to eat the rats when they get a chance. Both the cats and the rats can only move with constant speed 1.
For each valid i, the i-th cat is initially sleeping at a point a_i. At a time s_i, this cat wakes up and starts moving to a final point b_i with constant velocity and without any detours (so it arrives at this point at the time e_i = s_i + |a_i-b_i|). After it arrives at the point b_i, it falls asleep again.
For each valid i, the i-th rat is initially hiding at a point c_i. At a time r_i, this rat stops hiding and starts moving to a final point d_i in the same way as the cats ― with constant velocity and without any detours, arriving at the time q_i = r_i + |c_i-d_i| (if it does not get eaten). After it arrives at the point d_i, it hides again.
If a ca

In [14]:
with open('problem_samples_after.json', 'w', encoding='utf-8') as f:
    json.dump(problems_after, f, indent=4, ensure_ascii=False)

In [17]:
problems_after = load_json('problem_samples_after.json')

In [26]:
print(problems_after['ENGLISH'])

Problem
Chef is learning English. So far, he knows N words, and he wants to write a verse.
Let's call a pair of English words a stanza. A verse is a list of stanzas. The prefixal rhyme of a stanza is the longest common prefix of the words in it. Similarly, the suffixal rhyme of a stanza is the longest common suffix of the words in it. The beauty of a stanza whose prefixal and suffixal rhymes have lengths l_p and l_s respectively is Min(l_p, l_s)^2.
For example, a stanza formed by the words "abcdefghijkl" and "abcxyzhijkl" has a prefixal rhyme "abc" (with length 3), suffixal rhyme "hijkl" (with length 5) and beauty Min(3,5)^2 = 3^2 = 9.
The beauty of a verse is the sum of beauties of all its stanzas. For example, the beauty of the verse (("abcdefghijkl", "abcxyzhijkl"), ("world", "word"), ("code", "chef")) is 9+1+0=10.
You are given N words W_1, W_2, ..., W_N (they are not necessarily distinct). Help Chef write the most beautiful verse. It is not necessary to use all the words, but each

In [5]:
with open('chatgpt_result_comparing_diff.json', 'r', encoding='utf-8') as f:
    chatgpt_res_compare_diff = json.load(f)

with open('chatgpt_result_comparing_same.json', 'r', encoding='utf-8') as f:
    chatgpt_res_compare_same = json.load(f)

In [6]:
for k, v in chatgpt_res_compare_diff.items():
    print(f'[{k}]\n')
    print(v['result'])
    print('\n----')

[CATSRATS_43]

The algorithm used in the solution code is simulation.

There is no significant difference between the solution code and the correct answer code. The code implements the correct algorithm to solve the problem.

----
[ANTIKNAPSACK_60]



The submitted solution code uses the following algorithm:
- Read the number of test cases.
- For each test case, read N and K.
- Find the smallest divisor of K between 1 and 19, and assign it to a variable 'res'.
- For each integer from 1 to N, print 'res' multiplied by the integer.

The answer code uses the same algorithm as the submitted solution code, except for one difference:
- In the loop to find the smallest divisor of K, the submitted solution code checks if K is divisible by 'i', while the answer code checks if K is NOT divisible by 'i'. This means that the answer code is looking for the smallest number between 1 and 19 that does not divide K, while the submitted solution code is looking for the smallest number that does divide K

In [7]:
problems = load_json('problem_samples_after.json')
singles = load_json('singles.json')
multis = load_json('multis.json')

solutions = singles.copy()
solutions.update(multis)

In [8]:
for k, v in chatgpt_res_compare_same.items():
    print(f'[{k}]\n')
    print(v['desc'])
    print('\n----')

[CATSRATS_43]

Problem
There are N cats (numbered 1 through N) and M rats (numbered 1 through M) on a line. Each cat and each rat wants to move from some point to some (possibly the same) point on this line. Naturally, the cats also want to eat the rats when they get a chance. Both the cats and the rats can only move with constant speed 1.
For each valid i, the i-th cat is initially sleeping at a point a_i. At a time s_i, this cat wakes up and starts moving to a final point b_i with constant velocity and without any detours (so it arrives at this point at the time e_i = s_i + |a_i-b_i|). After it arrives at the point b_i, it falls asleep again.
For each valid i, the i-th rat is initially hiding at a point c_i. At a time r_i, this rat stops hiding and starts moving to a final point d_i in the same way as the cats ― with constant velocity and without any detours, arriving at the time q_i = r_i + |c_i-d_i| (if it does not get eaten). After it arrives at the point d_i, it hides again.
If a

In [9]:
print(chatgpt_res_compare_diff['ZOOZ_565']['solution'])

# cook your dish here
try:
    for _ in range(int(input())):
        n = int(input())
        std_str = "1"
        
        if n==3:
            print("010")
        if n==4:
            print("1001")
            
        else:
            if n%2==0:
                std_str+=((n-2)//2)*'01'
                std_str+='1'
            else:
                std_str='0'+(n//2)*'10'
            print(std_str)
            
except:
    pass


In [10]:
print(chatgpt_res_compare_diff['ZOOZ_565']['answer'])

# cook your dish here
try:
    for _ in range(int(input())):
        n = int(input())
        if n%2==0:
            ans ='1'+(n-2)*'0' + '1'
        else:
            ans ='0'+(n-2)*'1' + '0'
        print(ans)
except:
    pass


In [11]:
print(chatgpt_res_compare_same['ZOOZ_565']['desc'])

Problem
Kulyash believes in equality.
Given an integer N, output a binary string of length N such that:
The count of 01 subsequences in the string is equal to the count of 10 subsequences;
The string has at least one occurrence of 0 as well as 1.
If multiple such strings exist, print any. Also, it is guaranteed that corresponding to the given input, an answer always exists.

Input Format
First line will contain T, number of test cases. Then the test cases follow.
Each test case contains of a single line of input, an integer N - the length of the binary string.

Output Format
For each test case, output any binary string of length N satisfying the given conditions.

Constraints
1 <= T <= 100
3 <= N <= 1000
Subtasks


In [12]:
def make_prompt(**kwargs):
    desc = kwargs['desc']
    solution = kwargs['solution']
    answer = kwargs['answer']
    prompt = f'''You are the algorithm analyzer who figures out the main differences of given wrong solution code from the answer code, who are correct to solve given problem description. You need to figure out the main differences of the algorithm which makes the solution code failed to solve the problem, comparing the answer code. There might be only simple typo error, or simple logical error. Regardless of the size of the differences, you must specify the differences of the wrong solution code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- You must specify the main differences is simple, like a typo error or simple logical error, or complex enough, such as inefficient algorithm or wrong approach, or existence of missing logic.
- If their time complexities are different although their algorithms are same, you can judge them as different algorithm.
- You can compare them line by line, but you need to compare them in abstracted level, so that I can compare them in terms of their main algorithm which the codes want to implement.
- When the differences of code lines are given, you must explain the differences of the algorithm considering the differences of code lines, and why that different lines make the different results.
- You must explain about the differences of the solution code only, not about the answer code. I want the result must not be include any direct explanation about the algorithm of the answer code, but only hints by giving a differences between them.
- You cannot describe the algorithm of the answer code directly. It must be hidden to viewer.


Here is the problem description:

{desc}


Here is the wrong solution code:

{solution}


Here is the answer code:

{answer}'''
    return prompt

In [13]:
from difflib import unified_diff
import sys

In [14]:
sys.stdout.writelines([x+'\n' for x in unified_diff(
    chatgpt_res_compare_diff['ZOOZ_565']['answer'].split('\n'),
    chatgpt_res_compare_diff['ZOOZ_565']['solution'].split('\n'),
)])

--- 

+++ 

@@ -2,10 +2,20 @@

 try:
     for _ in range(int(input())):
         n = int(input())
-        if n%2==0:
-            ans ='1'+(n-2)*'0' + '1'
+        std_str = "1"
+        
+        if n==3:
+            print("010")
+        if n==4:
+            print("1001")
+            
         else:
-            ans ='0'+(n-2)*'1' + '0'
-        print(ans)
+            if n%2==0:
+                std_str+=((n-2)//2)*'01'
+                std_str+='1'
+            else:
+                std_str='0'+(n//2)*'10'
+            print(std_str)
+            
 except:
     pass


In [16]:
print(make_prompt(**chatgpt_res_compare_diff['ZOOZ_565']))

You are the algorithm analyzer who figures out the main differences of given wrong solution code from the answer code, who are correct to solve given problem description. You need to figure out the main differences of the algorithm which makes the solution code failed to solve the problem, comparing the answer code. There might be only simple typo error, or simple logical error. Regardless of the size of the differences, you must specify the differences of the wrong solution code.

You must follow these instruction when you compare solution code and answer code.

- You need to tell me the name of the algorithm the code uses, both of solution code and answer code, if the name exists. When the name of the algorithms of two codes are different, the codes are different.
- You must specify the main differences is simple, like a typo error or simple logical error, or complex enough, such as inefficient algorithm or wrong approach, or existence of missing logic.
- If their time complexities a